In [20]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

import environment
import dqn
import per

In [21]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,10):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

jamming_bandwidths = [10e6, 20e6, 30e6]
jamming_fracs_array = np.array([])


In [22]:
def update_ddqn(shared_dqn, target_dqn, optimizer, replay_buffer, batch_size, gamma, beta):
    if replay_buffer.size() < batch_size:
        return
    
    #sample from prioritized replay buffer
    states, actions, rewards, next_states, dones, indices, weights = replay_buffer.sample(batch_size, beta)
    
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions).unsqueeze(1)
    rewards = torch.FloatTensor(rewards).unsqueeze(1)
    next_states = torch.FloatTensor(next_states)
    dones = torch.FloatTensor(dones).unsqueeze(1)
    weights = torch.FloatTensor(weights).unsqueeze(1)

    #current q-values
    q_values = shared_dqn(states).gather(1, actions)

    #ddqn update rule
    next_actions = shared_dqn(next_states).argmax(1, keepdim=True)
    next_q_values = target_dqn(next_states).gather(1, next_actions)

    target_q_values = rewards + gamma * next_q_values * (1 - dones)

    #weighted MSE loss with importance sampling correction
    td_errors = target_q_values - q_values
    loss = (weights * td_errors ** 2).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #update priorities based on new TD errors
    new_priorities = td_errors.abs().detach().numpy() + 1e-5
    replay_buffer.update_priorities(indices, new_priorities)


In [23]:
#select action
def select_action(state, epsilon, action_size, model):
    if np.random.rand() < epsilon:
        return np.random.randint(action_size) 
    else:
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            q_values = model(state_tensor)
            return q_values.argmax().item()

#update target fn
def update_target(main_dqn, target_dqn):
    target_dqn.load_state_dict(main_dqn.state_dict())

In [ ]:
def train_ddqn_per(env, num_episodes, batch_size, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995, 
                   beta_start=0.4, beta_end=1.0, beta_increment_per_episode=0.001, target_update_freq=10, jamming_fracs_array = jamming_fracs_array):

    agent_id = env.agents[0]

    obs_size = env.observation_space(agent_id).shape[0]
    action_size = env.action_space[agent_id].n

    shared_dqn = dqn.DQN(obs_size, action_size)
    target_dqn = dqn.DQN(obs_size, action_size)
    target_dqn.load_state_dict(shared_dqn.state_dict())

    optimizer = optim.Adam(shared_dqn.parameters(), lr=1e-4)

    replay_buffer = per.PrioritizedReplayBuffer(10000)

    epsilon = epsilon_start
    beta = beta_start

    total_rewards = []
    mean_rewards = []

    plt.ion()
    plt.figure(figsize=(10, 5))

    for episode in range(num_episodes):
        states = env.reset()
        episode_rewards = {agent: 0 for agent in env.agents}
        done = {agent: False for agent in env.agents}

        while not all(done.values()):
            actions = {}
            for agent in env.agents:
                if not done[agent]:
                    actions[agent] = select_action(states[agent], epsilon, action_size, shared_dqn)

            #env step
            next_states, rewards, dones, _ = env.step(actions)

            #add episode to replay buffer
            for agent in env.agents:
                replay_buffer.add(states[agent], actions[agent], rewards[agent], next_states[agent], dones[agent])
                episode_rewards[agent] += rewards[agent]

            # Update DDQN
            update_ddqn(shared_dqn, target_dqn, optimizer, replay_buffer, batch_size, gamma, beta)

            states = next_states
            done = dones

            #add jamming fractions for each episode
        if all(done.values()):
            if episode == 0:
                jamming_fracs_array = env.jamming_fractions
            else:
                episode_jamming_fractions = env.jamming_fractions
                jamming_fracs_array = np.vstack((jamming_fracs_array,episode_jamming_fractions))


        total_reward = sum(episode_rewards.values())
        total_rewards.append(total_reward)

        #calculate mean rewards for every 100 episodes
        if (episode + 1) % 100 == 0:
            mean_reward = np.mean(total_rewards[-100:])
            mean_rewards.append(mean_reward)
        else:
            mean_rewards.append(mean_rewards[-1] if mean_rewards else 0)

        #decay epsilon and update beta
        epsilon = max(epsilon_end, epsilon * epsilon_decay)
        beta = min(beta_end, beta + beta_increment_per_episode)

        #update target dqn
        if episode % target_update_freq == 0:
            update_target(shared_dqn, target_dqn)

        print(f'Episode {episode + 1}: Total reward = {total_reward}')
        #save the weights every 1000 episodes and on the best episode
        if episode % 1000 == 0 or total_reward == max(total_rewards) and total_reward > 0:
            torch.save(shared_dqn.state_dict(), f'ddqn_weights_{episode}.pth')

        #save the best weights
        if total_reward == max(total_rewards) and total_reward > 0:
            torch.save(shared_dqn.state_dict(), 'ddqn_best_weights.pth')

    plt.ioff()
    plt.show()

    return shared_dqn, jamming_fracs_array

In [24]:
env = environment.ParallelRadarJammingEnv(hop_states, jamming_bandwidths)
ddqn, jamming_fracs_array = train_ddqn_per(env, num_episodes=8000, 
                batch_size=128,
                gamma=0.90, 
                epsilon_start=1, 
                epsilon_end=0.05, 
                epsilon_decay=0.999,
                beta_start=0.3,
                beta_end=1.0, 
                beta_increment_per_episode=0.00005, 
                target_update_freq=40)

Episode 1: Total reward = -38720
Episode 2: Total reward = -43390


KeyboardInterrupt: 

<Figure size 1000x500 with 0 Axes>

In [15]:
#split jamming_fracs array into 3 arrays for each radar
jamming_fracs_array_0 = jamming_fracs_array[:,0]
jamming_fracs_array_1 = jamming_fracs_array[:,1]
jamming_fracs_array_2 = jamming_fracs_array[:,2]

#take the max element per every 100 elements of each array, and add them to a new array
max_jamming_fracs_array_0 = np.array([np.max(jamming_fracs_array_0[i:i+100]) for i in range(0, len(jamming_fracs_array_0), 100)])
max_jamming_fracs_array_1 = np.array([np.max(jamming_fracs_array_1[i:i+100]) for i in range(0, len(jamming_fracs_array_1), 100)])
max_jamming_fracs_array_2 = np.array([np.max(jamming_fracs_array_2[i:i+100]) for i in range(0, len(jamming_fracs_array_2), 100)])

In [ ]:
sns.lineplot(jamming_fracs_array_0, label='Radar 0')
sns.lineplot(jamming_fracs_array_1, label='Radar 1')
sns.lineplot(jamming_fracs_array_2, label='Radar 2')
plt.xlabel('Episode')
plt.ylabel('Jamming Fractions')
plt.legend()
plt.show()

NameError: name 'jamming_fracs_array_0' is not defined

In [23]:
#test the trained model with some epsilon greedy policy
def test_model(env, model, num_episodes=10):

    agent_id = env.agents[0]
    obs_size = env.observation_space(agent_id).shape[0]
    action_size = env.action_space[agent_id].n

    total_rewards = []

    for episode in range(num_episodes):
        states = env.reset()
        episode_rewards = {agent: 0 for agent in env.agents}
        done = {agent: False for agent in env.agents}

        while not all(done.values()):
            actions = {}
            for agent in env.agents:
                if not done[agent]:
                    state = torch.FloatTensor(states[agent]).unsqueeze(0)
                    q_values = model(state)
                    actions[agent] = q_values.argmax().item()

            next_states, rewards, dones, _ = env.step(actions)

            for agent in env.agents:
                episode_rewards[agent] += rewards[agent]

            states = next_states
            done = dones

        total_reward = sum(episode_rewards.values())
        total_rewards.append(total_reward)

        print(f'Episode {episode + 1}: Total reward = {total_reward}')

    return total_rewards

In [33]:
test_radar = [[5 * math.pow(10, 9), 5.4* math.pow(10, 9)], [4 * math.pow(10, 9), 4.5* math.pow(10, 9)], [7 * math.pow(10, 9), 7.2* math.pow(10, 9)]]

hop_states_test = []

for i in range (0,3):
    radar_states_test = []
    interval_test = (test_radar[i][1] - test_radar[i][0])/10
    for j in range(0,10):
        radar_states_test.append(int(test_radar[i][0]+ interval_test * j))
    hop_states_test.append(radar_states_test)


In [34]:
#load the best weights and test the model

ddqn.load_state_dict(torch.load('ddqn_weights_3632.pth'))
ddqn.eval()
env = environment.ParallelRadarJammingEnv(hop_states_test, jamming_bandwidths, max_hop_length=10)
rewards = test_model(env, ddqn, num_episodes=50)

print(f'Mean reward = {np.mean(rewards)}')

Episode 1: Total reward = 72600
Episode 2: Total reward = 87180
Episode 3: Total reward = 86800
Episode 4: Total reward = 76380
Episode 5: Total reward = 76380
Episode 6: Total reward = 55890
Episode 7: Total reward = 72600
Episode 8: Total reward = 87020
Episode 9: Total reward = 87000
Episode 10: Total reward = 68110
Episode 11: Total reward = 86810
Episode 12: Total reward = 69180
Episode 13: Total reward = 62360
Episode 14: Total reward = 62180
Episode 15: Total reward = 39890
Episode 16: Total reward = 86980
Episode 17: Total reward = 76200
Episode 18: Total reward = 61800
Episode 19: Total reward = 65400
Episode 20: Total reward = 75490
Episode 21: Total reward = 58400
Episode 22: Total reward = 86980
Episode 23: Total reward = 83200
Episode 24: Total reward = 55510
Episode 25: Total reward = 44380
Episode 26: Total reward = 69180
Episode 27: Total reward = 79960
Episode 28: Total reward = 72600
Episode 29: Total reward = 40420
Episode 30: Total reward = 62710
Episode 31: Total r